In [2]:
import requests
import json
import os


In [ ]:
#Input API Key here , is a String
api_key = None #Register a account at RapidAPI for one

In [ ]:
listNames=[]
listReviews=[]
listOverallRatings=[]
listUserRatings=[]
listGenre=[]
listId=[]
listIsVideo=[]
listReleaseDate=[]
listProductionCompany=[]

In [3]:
def saveToJson(content,name,type): 
    # Convert the JSON data into a Python object
    data = json.loads(content)

    # Specify the file path where you want to save the JSON file
    file_path = os.getcwd() +"/queriedMovieDetails/" + f"{name}" + f"{type}" + ".json"

    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Open the file in write mode and save the JSON data
    with open(file_path, "w") as outfile:
        json.dump(data, outfile)

**This saves all movie results from the single letter query as a json file each file representing the data for 1 movie result**

In [26]:
#Get Movie names from binary search query a,e,i,r,o
url = "https://flixster.p.rapidapi.com/search"
queryStrings = ["e","i","r","o","a","t","n","s","l","c"]
queryStrings = ["e"]

for query in queryStrings:
	querystring = {"query":query,"zipCode":"90002","radius":"50"}

	headers = {
		"X-RapidAPI-Key": api_key,
		"X-RapidAPI-Host": "flixster.p.rapidapi.com"
	}

	response = requests.request("GET", url, headers=headers, params=querystring)
	content = response.text
	saveToJson(content,query,"Details")


**This method queries Flixster for the review data for a particular movie using it's emsId**

In [ ]:
def getMovieReviewsByEmsId(emsId):
	#Using movie results from the binary search, GET the reviews with the respective emsId
	url = "https://flixster.p.rapidapi.com/reviews/list"

	querystring = {"emsId":{emsId},"limit":"20","offset":"0","withComments":"true"}

	headers = {
		"X-RapidAPI-Key": api_key,
		"X-RapidAPI-Host": "flixster.p.rapidapi.com"
	}

	response = requests.request("GET", url, headers=headers, params=querystring)
	return response


In [ ]:
def getMoreMovieDetailsByEmsVersionId(emsVersionId):
    url = "https://flixster.p.rapidapi.com/movies/detail"

    querystring = {"emsVersionId":emsVersionId}

    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "flixster.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response


**Below, we iterate through all movie data json we scraped from the single letter query and extract it's emsId and emsVersionId. Using the extracted Ids, we can then scrape the relevant Movie Review data and additional movie details from Flixster**

In [ ]:
#After getting queried Movie results
#Read all the emsId from the results Json
# set the path to the folder containing the JSON files
folder_path = os.getcwd() +"/queriedMovieDetails/"

# loop through all files in the folder
for file_name in os.listdir(folder_path):
    # check if file is a JSON file
    if file_name.endswith('.json'):
        # open the file and load the JSON data
        with open(os.path.join(folder_path, file_name)) as json_file:
            data = json.load(json_file)
            
            # do something with the JSON data, such as extracting specific values
            movie_name = data['data']['movie']['name']
            movie_ems = data['data']['movie']['emsId']
            movie_ems_versionId = data['data']['movie']['emsVersionId'] #Another id used to get more movie detail data e.g Release date, genre ....
            
            # print the values for this file
            review_result = getMovieReviewsByEmsId(movie_ems)
            list_of_reviews = review_result['data']['audienceReviews']['items']
            if (len(list_of_reviews)!=0):
                for review in list_of_reviews:
                    if (review['rating']==None or review['comment']==None):
                        break
                    listNames.append(movie_name)
                    listId.append(movie_ems)
                    listUserRatings.append(review['rating'])
                    listReviews.append(review['comment'])
                    listIsVideo.append("FALSE")
                    more_movie_data = getMoreMovieDetailsByEmsVersionId(movie_ems_versionId)['data']['movie']
                    movie_score = more_movie_data['userRating']['dtlLikedScore']
                    if (score!=None):
                        listOverallRatings.append(movie_score)
                    else:
                        listOverallRatings.append(5)
                    listReleaseDate.append(more_movie_data['releaseDate'])
                    #Logic to retrieve genres of the movie and clean and convert to a string
                    genre_string = ""
                    current_movie_genre = more_movie_data['genres']
                    #Logic for retrieving genres
                    for i in range(len(current_movie_genre)):
                        if i!=len(current_movie_genre)-1:
                            genre_string+=f"{current_movie_genre[i]['name']},"
                        else:
                            genre_string+=f"{current_movie_genre[i]['name']}"
                    listGenre.append(genre_string)
                    genre_string=""
                    listProductionCompany.append("Unknown")
                pass


**Creating a pd.df with scraped data and save as CSV**

In [ ]:
data = {
    'Movie Name': listNames,
    'Movie Id' : listId,
    'Review Content': listReviews,
    'tmdb Rating' : listOverallRatings,
    'User Rating': listUserRatings,
    'Genre(s)' : listGenre,
    'Not A Movie?' : listIsVideo,
    'Release Date' : listReleaseDate,
    'Production Company' : listProductionCompany
}

# Create a Pandas dataframe from the data
df = pd.DataFrame(data)

# Save the dataframe to a CSV file
#df.to_csv('test_clean_data_extended_newest_data.csv', index=True)